# Init

In [24]:
from pandas_data_reader_service_core.modules.finam.search_service import Search, Market
from pandas_data_reader_service_core.modules.finam.stock_info import FinamStockInfo
from pandas_data_reader_service_core.service import PandasDataReaderService as pdrs, StockInfo, TimeFrame

from datetime import date

import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

In [25]:
stock = "EURUSD"
market = Market.CURRENCIES_WORLD

date_from = date(2020,1,7)
date_till = date(2021,7,14)
tf = TimeFrame.DAILY

# Load data

In [26]:
stock_list = Search.by_code(stock, market)
stock_list

,name,code,market
id,,,
83,Eur/Usd,EURUSD,5


In [27]:
stock_info = StockInfo.init_for_Finam(FinamStockInfo(stock_list.index[0]))
print(stock_info)

Source StockSource.Finam source StockInfo Market 5 code EURUSD index 83


In [28]:
data_df = pdrs().get(stock=stock_info,  date_from=date_from,date_to=date_till,time_frame=tf,short_col_name=True)
data_df

,O,H,L,C,V
DT,,,,,
2020-01-07,1.11954,1.11971,1.11320,1.11540,1610661
2020-01-08,1.11541,1.11682,1.11000,1.11127,2131727
2020-01-09,1.11129,1.11204,1.10920,1.11092,1423627
2020-01-10,1.11093,1.11292,1.10840,1.11200,1299383
2020-01-12,1.11175,1.11225,1.11108,1.11160,34731
...,...,...,...,...,...
2021-07-11,1.18500,1.18785,1.18500,1.18724,43118
2021-07-12,1.18724,1.18802,1.18340,1.18626,776966
2021-07-13,1.18619,1.18751,1.17700,1.17740,925275


In [29]:
fig = go.Figure(data=[go.Candlestick(x=data_df.index,
                open=data_df['O'],
                high=data_df['H'],
                low=data_df['L'],
                close=data_df['C'])])
fig.update_yaxes(fixedrange=False)
fig.update_layout(height=1000)
fig.show()

# Find Rules

## Support functions

In [30]:
from __future__ import annotations

class candle_chart:
    def __init__(self,df: pd.DataFrame) -> None:
        self.fig = go.Figure(data=[go.Candlestick(x=df.index,
            open= df['O'],
            high= df['H'],
            low=  df['L'],
            close=df['C'])])
        self.fig.update_yaxes(fixedrange=False)
        self.fig.update_layout(height=1000)
        self.max = df.H.max()
        self.min = df.L.min()
        pass

    def add_marker(self, marker_candle:pd.Series)->candle_chart:
        self.fig.add_shape(type="line",
            x0=marker_candle.name, x1=marker_candle.name, y0=self.min, y1=marker_candle["L"],
            line=dict(
                color="MediumPurple",
                width=4,
                dash="dot",
            )
        )

        self.fig.add_shape(type="line",
            x0=marker_candle.name, x1=marker_candle.name, y0=marker_candle["H"], y1=self.max,
            line=dict(
                color="MediumPurple",
                width=4,
                dash="dot",
            )
        )
        return self
    
    def add_wave_from_point(self, start_point: point, end_point:point, main_wave: bool = True)->candle_chart:
        return self.add_wave(start_point.dt,start_point.value,end_point.dt,end_point.value,main_wave)
        
    def add_wave(self, start_dt:pd.Timestamp, start_value:float, end_dt:pd.Timestamp, end_value:float, main_wave: bool = True)->candle_chart:
        if end_value > start_value:
            color = "Green"
        else:
            color = "Red"
        if main_wave:
            dash = "solid"
        else:
            dash = "dashdot"
        self.fig.add_shape(type="line",
            x0=start_dt, x1=end_dt, y0=start_value, y1=end_value,
            line=dict(
                color=color,
                width=2,
                dash=dash,
            )
        )
        return self
    def show(self):
        self.fig.show()

class point:
    def __init__(self,dt:pd.Timestamp, value:float) -> None:
        self.dt = dt
        self.value = value
        pass
    @staticmethod
    def from_candle(candle:pd.Series, value_name:str)->point:
        return point(candle.name, candle[value_name])

def height(point_start:point, point_end:point ):
    return point_end.value - point_start.value

def check_rule(rule_name: str, answer:bool):
    if not answer:
        raise Exception(f"Rule {rule_name} doesn't passed")
    else:
        print(f"Rule {rule_name}: checked")

In [31]:
zoom = (pd.Timestamp(year=2020, month=5,day=1),  pd.Timestamp(year=2021,month=2,day=1))
zoom_df = data_df.loc[[dt for dt in data_df.index if (dt >= zoom[0]) & (dt <= zoom[1])]]

## Wave1

### Define wave

#### Define Start Wave 1 SW1

In [32]:
SW1_dt = pd.Timestamp(year=2020,month=5,day=14)
SW1_dt

Timestamp('2020-05-14 00:00:00')

In [33]:
SW1_candle = data_df.loc[SW1_dt]
SW1_candle

O    1.082110e+00
H    1.082400e+00
L    1.077300e+00
C    1.080560e+00
V    1.696707e+06
Name: 2020-05-14 00:00:00, dtype: float64

#### Define End Wave 1 EW1

In [34]:
EW1_dt = pd.Timestamp(year=2020,month=6,day=10)
EW1_dt

Timestamp('2020-06-10 00:00:00')

In [35]:
EW1_candle = data_df.loc[EW1_dt]
EW1_candle

O    1.133160e+00
H    1.142250e+00
L    1.132050e+00
C    1.136650e+00
V    2.821155e+06
Name: 2020-06-10 00:00:00, dtype: float64

#### Define range wave 1 RW1

In [36]:
RW1 = zoom_df.loc[[dt for dt in zoom_df.index if (dt >= SW1_dt) & (dt <= EW1_dt)]]
RW1.head()

,O,H,L,C,V
DT,,,,,
2020-05-14,1.08211,1.08240,1.07730,1.08056,1696707
2020-05-15,1.08049,1.08512,1.07875,1.08179,1528162
2020-05-17,1.08380,1.08380,1.08060,1.08185,72782
2020-05-18,1.08187,1.09270,1.07980,1.09137,1630550
2020-05-19,1.09139,1.09761,1.09006,1.09260,1629331


#### Define values

In [37]:
SW1_point = point.from_candle(SW1_candle,"L")
EW1_point = point.from_candle(EW1_candle,"H")

In [38]:
candle_chart(zoom_df).add_marker(SW1_candle).add_marker(EW1_candle).add_wave_from_point(SW1_point,EW1_point).show()

### Rules

In [60]:
check_rule("End wave 1 > start wave 1", EW1_point.value > SW1_point.value) 
check_rule("EW1 is max on range W1", (RW1[["H","L"]] < EW1_candle.H).any(axis=1).any())

Rule End wave 1 > start wave 1: checked
Rule EW1 is max on range W1: checked


## Wave 2

### Define wave

#### Define Start Wave 2 SW2

In [40]:
SW2_dt = EW1_dt
SW2_candle = zoom_df.loc[SW2_dt]
SW2_candle

O    1.133160e+00
H    1.142250e+00
L    1.132050e+00
C    1.136650e+00
V    2.821155e+06
Name: 2020-06-10 00:00:00, dtype: float64

#### Define End Wave 2 SW2

In [41]:
EW2_dt = pd.Timestamp(year=2020,month=6,day=19)
EW2_candle = zoom_df.loc[EW2_dt]
EW2_candle

O    1.120720e+00
H    1.125440e+00
L    1.116600e+00
C    1.117600e+00
V    2.234771e+06
Name: 2020-06-19 00:00:00, dtype: float64

#### Define range wave 2 RW2

In [42]:
RW2 = zoom_df.loc[[dt for dt in zoom_df.index if (dt >= SW2_dt) & (dt <= EW2_dt)]]
RW2.head()

,O,H,L,C,V
DT,,,,,
2020-06-10,1.13316,1.14225,1.13205,1.13665,2821155
2020-06-11,1.13667,1.14037,1.12850,1.12865,3285237
2020-06-12,1.12876,1.13406,1.12110,1.12601,2897746
2020-06-14,1.12480,1.12587,1.12250,1.12514,159708
2020-06-15,1.12514,1.13384,1.12250,1.13373,3251692


#### Define values

In [43]:
SW2_point = point.from_candle(SW2_candle,"H")
EW2_point = point.from_candle(EW2_candle,"L")

In [45]:
candle_chart(zoom_df).add_marker(SW1_candle).add_marker(EW1_candle).add_wave_from_point(SW1_point,SW2_point)\
                     .add_marker(SW2_candle).add_marker(EW2_candle).add_wave_from_point(SW2_point,EW2_point,main_wave=False)\
                     .show()

### Rules

In [48]:
check_rule("End wave 2 < start start wave 2",EW2_point.value < SW2_point.value)
check_rule("EW2 is min on RW2", (RW2[["H","L"]] > EW2_candle.L).any(axis=1).any())
check_rule("End wave 2 > Start wave 1",EW2_point.value > SW1_point.value)


Rule End wave 2 < start start wave 2: checked
Rule EW2 is min on RW2: checked
Rule End wave 2 > Start wave 1: checked


## Wave 3

### Define wave

#### Define Start (SW3)

In [54]:
SW3_dt = EW2_dt
SW3_candle = zoom_df.loc[SW3_dt]
SW3_candle

O    1.120720e+00
H    1.125440e+00
L    1.116600e+00
C    1.117600e+00
V    2.234771e+06
Name: 2020-06-19 00:00:00, dtype: float64

#### Define End (EW3)

In [52]:
EW3_dt = pd.Timestamp(year=2020,month=9,day=1)
EW3_candle = zoom_df.loc[EW3_dt]
EW3_candle

O    1.193650e+00
H    1.201140e+00
L    1.190000e+00
C    1.192230e+00
V    1.983008e+06
Name: 2020-09-01 00:00:00, dtype: float64

#### Define range (RW3)

In [55]:
RW3 = zoom_df.loc[[dt for dt in zoom_df.index if (dt >= SW3_dt) & (dt <= EW3_dt)]]
RW3.head()

,O,H,L,C,V
DT,,,,,
2020-06-19,1.12072,1.12544,1.1166,1.11760,2234771
2020-06-21,1.11900,1.11940,1.1167,1.11764,93933
2020-06-22,1.11765,1.12810,1.1172,1.12778,1979244
2020-06-23,1.12776,1.13486,1.1231,1.13124,2482232
2020-06-24,1.13121,1.13258,1.1246,1.12497,2216060


#### Define vales

In [58]:
SW3_point = point.from_candle(SW3_candle,"L")
EW3_point = point.from_candle(EW3_candle,"H")
HW3 = EW3

In [59]:
candle_chart(zoom_df).add_marker(SW1_candle).add_marker(EW1_candle).add_wave_from_point(SW1_point,SW2_point)\
                     .add_marker(SW2_candle).add_marker(EW2_candle).add_wave_from_point(SW2_point,EW2_point,main_wave=False)\
                     .add_marker(SW3_candle).add_marker(EW3_candle).add_wave_from_point(SW3_point,EW3_point)\
                     .show()

### Rules

In [62]:
check_rule("End wave 3 > start wave 3", EW3_point.value > SW3_point.value) 
check_rule("EW3 is max on range W3", (RW3[["H","L"]] < EW3_candle.H).any(axis=1).any())
check_rule("End wave 3 > start wave 1", EW3_point.value > EW1_point.value) 
check_rule("Height wave 3 > Height wave 1", )

Rule End wave 3 > start wave 3: checked
Rule EW3 is max on range W3: checked
Rule End wave 3 > start wave 1: checked
